# Рынок вакансий для дата аналитика или BI аналитика джуна в Европе

**Цель:** Визуализировать информацию о рынке вакансий для дата аналитика или BI аналитика джуна в Европе

**Источник данных:** файл сsv c вакансиями за неделю, спарсенными с LinkedIn 07/09/2022

**Техническое задание:**
1. Распарсить предоставленный csv файл с помощью BS 4, создав следующие признаки:
- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию

2. Подготовка данных к визуализации:
- фильтрация датафрейма с оставлением только вакансий для аналитиков данных и BI аналитиков
- удаление дубликатов
- удаление ненужных атрибутов (признаков)

3. Визуализация данных:
* [LinkedIn DA vacancies](https://app.powerbi.com/view?r=eyJrIjoiNjRjNTUyMzItZWI5Yy00NzY2LWIxNGUtODA5M2RlODc3NWI3IiwidCI6IjY2NzE2NDY3LTQ3OGMtNDQ1MC1hNTEyLTIxYjlhMzQwZmEyNSIsImMiOjEwfQ%3D%3D&pageName=ReportSection)

## Получение и обработка данных

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('masterskaya_yandex_2022_09_07.csv')

### Название вакансии

In [4]:
data['title'] = data['html'].apply(lambda x:  BeautifulSoup(x).find('h2').text.strip())

In [5]:
f"Количество пропусков: {data['title'].isna().sum()}"

'Количество пропусков: 0'

**Добавим ссылки**

In [6]:
data['link'] = data['html'].apply(lambda x: "https://linkedin.com" + BeautifulSoup(x).find('a').get('href'))

### Количество кандидатов

In [7]:
def get_candidats(cell):
    try:
        return BeautifulSoup(cell).find(
            'span', class_='jobs-unified-top-card__applicant-count'
        ).text.strip().replace(' applicants', '')
    except:
        return np.nan

In [8]:
data['candidats'] = data['html'].apply(get_candidats)

In [9]:
data['candidats'] = pd.to_numeric(data['candidats'], errors='coerce')

In [10]:
f"Количество пропусков: {data['candidats'].isna().sum()}"

'Количество пропусков: 125'

В некоторых вакансиях не указано количество кандидатов, так как на момент парсинга еще никто не откликнулся.

### Страна и город

In [11]:
def get_geo(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip()
    except:
        return np.nan

In [12]:
data['geo'] = data['html'].apply(get_geo)

In [13]:
def get_city(cell):
    if len(cell.split(',')) > 1:
        return cell.split(',')[0].strip()
    elif "Metropolitan" in cell or "Greater" in cell:
        return cell.replace('Greater', '').replace('Metropolitan', '').replace('Area', '').replace('Region','').strip()
    else:
        return np.nan

In [14]:
data['city'] = data['geo'].apply(get_city)

In [15]:
def get_country(cell):
    if len(cell.split(',')) > 1:
        return cell.split(',')[-1].strip()
    elif "Metropolitan" in cell or "Greater" in cell or "Region" in cell:
        return np.nan
    else:
        return cell

In [16]:
data['country'] = data['geo'].apply(get_country)

In [17]:
data['country'].unique()

array(['France', 'Sweden', 'Belgium', 'Germany', nan, 'Hungary', 'Spain',
       'Denmark', 'Gibraltar', 'Poland', 'United Kingdom', 'Finland',
       'Switzerland', 'Netherlands', 'Italy', 'Norway', 'Czechia',
       'Bulgaria', 'Slovakia', 'Portugal', 'Ireland', 'Luxembourg',
       'Romania', 'Croatia', 'Slovenia', 'Greece', 'Austria', 'Estonia',
       'Lithuania', 'Ukraine', 'Italy Metropolitan Area', 'Serbia'],
      dtype=object)

Поправим название Италии

In [18]:
data['country'] = data['country'].replace('Italy Metropolitan Area', 'Italy')

In [19]:
f"Количество пропусков: {data['country'].isna().sum()}"

'Количество пропусков: 46'

Посмотрим на пропуски: сгруппируем по городам записи, где отсутствует название страны.

In [20]:
data[data['country'].isna()].groupby('city')['city'].count()

city
Antwerp       1
Athens        1
Barcelona     3
Berlin        1
Bologna       1
Bruges        1
Brussels      2
Bucharest     1
Cáceres       1
Edinburgh     1
Gdansk        1
Genoa         1
Ghent         2
Gijón         2
Grudziadz     1
Kortrijk      1
Liege         1
Lodz          2
Louvain       1
Milan         1
Modena        1
Mons          1
Namur         1
Nantes        1
Norrköping    1
Paris         3
Przemyśl      1
Radom         1
Rome          1
Turin         1
Valencia      1
Warsaw        1
Wroclaw       2
Zamosc        1
Zurich        1
Name: city, dtype: int64

Заполним пропуски.

In [21]:
cntrs = {'Poland':['Wroclaw','Gdansk','Grudziadz','Lodz','Przemyśl','Radom','Warsaw','Zamosc'], 
         'Italy':['Bologna','Genoa','Milan','Modena','Rome','Turin','Valencia'], 
         'Spain':['Barcelona','Cáceres','Gijón'],
         'Belgium':['Antwerp','Bruges','Brussels','Ghent','Kortrijk','Liege','Louvain','Mons','Namur'],
         'Greece': ['Athens'], 'Germany': ['Berlin'], 'Romania':['Bucharest'], 'Scotland':['Edinburgh'], 
         'France':['Nantes','Paris'], 'Sweden': ['Norrköping'], 'Switzerland':['Zurich']}

In [22]:
def fill_counties(row):
    if str(row['country']) == 'nan':
        city = row['city']
        for k, v in cntrs.items():
            if city in v:
                return k
    return row['country']

In [23]:
data['country'] = data.apply(fill_counties, axis=1)

In [24]:
data[data['country'].isna()]

,Unnamed: 0,html,title,link,candidats,geo,city,country
424,424,"\n <div>\n <div class=""\n jobs-deta...",Speech Data Collection in Germany,https://linkedin.com/jobs/view/3247818801/?alt...,NaN,Cologne Bonn Region,NaN,NaN
671,671,"\n <div>\n <div class=""\n jobs-deta...",Duales Masterstudium Applied Data Science (m/w/d),https://linkedin.com/jobs/view/3195587554/?alt...,7.0,Hannover-Braunschweig-Göttingen-Wolfsburg Region,NaN,NaN


Осталось два пропуска, заменим вручную.

In [25]:
data['country'][424] = 'Germany'
data['country'][671] = 'Germany'

In [26]:
f"Количество пропусков: {data['city'].isna().sum()}"

'Количество пропусков: 64'

In [77]:
data[data['city'].isna()].groupby('workplace')['link'].count()

workplace
On-site     1
Remote     62
Name: link, dtype: int64

In [82]:
data[data['city'].isna()].query('workplace == "On-site"')['link']

671    https://linkedin.com/jobs/view/3195587554/?alt...
Name: link, dtype: object

Некоторые вакансии с удаленкой не содержат город. В одной ваканчии он просто не указан, хотя позиция в офисе.

### Хард скилы

In [27]:
data['description'] = data['html'].apply(lambda x: BeautifulSoup(x).find('div', {'id':'job-details'}).text.strip())

In [28]:
skills = (['datahub', 'api', 'github', 'google analytics', 'adobe analytics', 'ibm coremetrics', 'omniture'
            'gitlab', 'erwin', 'hadoop', 'spark', 'hive'
           'databricks', 'aws', 'gcp', 'azure','excel',
            'redshift', 'bigquery', 'snowflake',  'hana'
            'grafana', 'kantar', 'spss', 
           'asana', 'basecamp', 'jira', 'dbeaver','trello', 'miro', 'salesforce', 
           'rapidminer', 'thoughtspot',  'power point',  'docker', 'jenkins','integrate.io', 'talend', 'apache nifi','aws glue','pentaho','google data flow',
             'azure data factory','xplenty','skyvia','iri voracity','xtract.io','dataddo', 'ssis',
             'hevo data','informatica','oracle data integrator','k2view','cdata sync','querysurge', 
             'rivery', 'dbconvert', 'alooma', 'stitch', 'fivetran', 'matillion','streamsets','blendo',
             'iri voracity','logstash', 'etleap', 'singer', 'apache camel','actian', 'airflow', 'luidgi', 'datastage',
           'python', 'vba', 'scala', ' r ', 'java script', 'julia', 'sql', 'matlab', 'java', 'html', 'c++', 'sas',
           'data studio', 'tableau', 'looker', 'powerbi', 'cognos', 'microstrategy', 'spotfire',
             'sap business objects','microsoft sql server', 'oracle business intelligence', 'yellowfin',
             'webfocus','sas visual analytics', 'targit', 'izenda',  'sisense', 'statsbot', 'panorama', 'inetsoft',
             'birst', 'domo', 'metabase', 'redash', 'power bi', 'alteryx', 'dataiku', 'qlik sense', 'qlikview'
          ])

In [29]:
def get_skills(cell):
    list_skills = []
    for skill in skills:
        if skill in cell.lower().replace('powerbi', 'power bi'):
            list_skills.append(skill)
    return list_skills

In [30]:
data['skills'] = data.description.apply(get_skills)

In [36]:
nans = 0
for row in data['skills']:
    if len(row) < 1:
        nans += 1
print(f"Количество пропусков: {nans}")        
    

Количество пропусков: 153


153 вакансии не имеют в описании требований к распространенным скиллам.

### Название компании

In [37]:
def get_company(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__company-name').text.strip()
    except:
        return np.nan

In [38]:
data['company'] = data['html'].apply(get_company)

In [39]:
f"Количество пропусков: {data['company'].isna().sum()}"

'Количество пропусков: 0'

### Тип занятости

In [40]:
def get_workplace(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__workplace-type').text.strip()
    except:
        return np.nan

In [41]:
data['workplace'] = data['html'].apply(get_workplace)

In [42]:
data['workplace'].unique()

array(['On-site', 'Remote', 'Hybrid', nan], dtype=object)

In [43]:
f"Количество пропусков: {data['workplace'].isna().sum()}"

'Количество пропусков: 131'

Много пропусков, в некоторых вакансиях действительно не указан тип занятости.

### Дата публикации вакансии

In [44]:
def get_date(cell):
    try:
        return BeautifulSoup(cell).find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
    except:
        return np.nan

In [45]:
data['date'] = data['html'].apply(get_date)

In [46]:
def date(cell):
    if 'minutes' in cell:
        for i in cell.split():
            if i.isdigit():
                cell = datetime(2022, 9, 7, 17) - timedelta(minutes=int(i))
                return cell
    if 'hours' in cell:
        for i in cell.split():
            if i.isdigit():
                cell = datetime(2022, 9, 7, 17) - timedelta(hours=int(i))
                return cell
    else:
        for i in cell.split():
            if i.isdigit():
                cell = datetime(2022, 9, 7, 17) - timedelta(days=int(i))
                return cell

In [47]:
data['date'] = data['date'].apply(date)

In [48]:
f"Количество пропусков: {data['date'].isna().sum()}"

'Количество пропусков: 0'

In [83]:
data.query('date < "2022-09-01 00:00"|(date > "2022-09-07 17:00")')

,Unnamed: 0,html,title,link,candidats,geo,city,country,description,skills,company,workplace,date,get_size_sphere,size,sphere


Все вакансии попадают в диапазон 01.09.2022 - 07.09.2022

### Размер компании (количество работников)

In [49]:
def get_size_sphere(cell):
    try:
        return BeautifulSoup(cell).find('div', class_ = 'mt5 mb2').find_all('li')[1].text.strip()
    except:
        return np.nan

In [50]:
data['get_size_sphere'] = data['html'].apply(get_size_sphere)

In [51]:
data[data['get_size_sphere'].isna()]

,Unnamed: 0,html,title,link,candidats,geo,city,country,description,skills,company,workplace,date,get_size_sphere
514,514,"\n <div>\n <div class=""\n jobs-deta...",Customer Ledger Data Analyst,https://linkedin.com/jobs/view/3254329032/?alt...,NaN,"Stockport, England, United Kingdom",Stockport,United Kingdom,"Salary: £25,000, plus performance bonus and be...","[excel, ssis, vba]",Birnbach Communications,On-site,2022-09-06 17:00:00,NaN
771,771,"\n <div>\n <div class=""\n jobs-deta...",Stage - Consultant(e) Data Engineer - Big Data...,https://linkedin.com/jobs/view/3241983304/?alt...,NaN,"Toulouse, Occitanie, France",Toulouse,France,,[],Avanade,NaN,2022-09-01 17:00:00,NaN


Есть два пропуска, заменим вручную.

In [52]:
data['get_size_sphere'][514] = '1-10 employees · Public Relations and Communications Services'
data['get_size_sphere'][771] = '10,001+ employees · IT Services and IT Consulting'

Добавим столбец с размером

In [53]:
def get_size(cell):
    if 'employees' in cell:
        cell = cell.partition(' employees')[0].replace(',', '')
        return cell 
    else:
        return np.nan

In [54]:
data['size'] = data['get_size_sphere'].apply(get_size)

In [55]:
f"Количество пропусков: {data['size'].isna().sum()}"

'Количество пропусков: 14'

In [85]:
data[data['size'].isna()]['link']

65     https://linkedin.com/jobs/view/3256339057/?alt...
247    https://linkedin.com/jobs/view/3257346745/?alt...
299    https://linkedin.com/jobs/view/3251175567/?alt...
364    https://linkedin.com/jobs/view/3254677314/?alt...
382    https://linkedin.com/jobs/view/3249619400/?alt...
400    https://linkedin.com/jobs/view/3247310439/?alt...
415    https://linkedin.com/jobs/view/3254305280/?alt...
526    https://linkedin.com/jobs/view/3249116636/?alt...
533    https://linkedin.com/jobs/view/3249828610/?alt...
545    https://linkedin.com/jobs/view/3249114970/?alt...
557    https://linkedin.com/jobs/view/3249913331/?alt...
607    https://linkedin.com/jobs/view/3247334292/?alt...
707    https://linkedin.com/jobs/view/3247325914/?alt...
765    https://linkedin.com/jobs/view/3249983776/?alt...
Name: link, dtype: object

В вакансиях рекрутинговых агенств и некоторых компаний не указан размер компаний, пропусков немного, это нормально.

### Сфера деятельности компании

In [56]:
def get_sphere(cell):
    if 'Premium' in cell:
        return np.nan
    if ('employees' in cell) and ('·' not in cell):
        return np.nan
    if '·' in cell:
        cell = cell.partition('· ')[2]
        return cell
    else:
        return cell

In [57]:
data['sphere'] = data['get_size_sphere'].apply(get_sphere)

In [58]:
f"Количество пропусков: {data['sphere'].isna().sum()}"

'Количество пропусков: 29'

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Unnamed: 0       772 non-null    int64         
 1   html             772 non-null    object        
 2   title            772 non-null    object        
 3   link             772 non-null    object        
 4   candidats        647 non-null    float64       
 5   geo              772 non-null    object        
 6   city             708 non-null    object        
 7   country          772 non-null    object        
 8   description      772 non-null    object        
 9   skills           772 non-null    object        
 10  company          772 non-null    object        
 11  workplace        641 non-null    object        
 12  date             772 non-null    datetime64[ns]
 13  get_size_sphere  772 non-null    object        
 14  size             758 non-null    object   

Распарсили все необходимые данные, по-возможности избавились от пропусков в данных.

## Подготовка данных к визуализации

**Удалим ненужные столбцы**

In [60]:
data.columns

Index(['Unnamed: 0', 'html', 'title', 'link', 'candidats', 'geo', 'city',
       'country', 'description', 'skills', 'company', 'workplace', 'date',
       'get_size_sphere', 'size', 'sphere'],
      dtype='object')

In [61]:
clear_data = data.drop(['Unnamed: 0', 'html', 'geo', 'description', 'get_size_sphere'], axis=1)

**Фильтрация вакансий**

In [62]:
clear_data = clear_data.query(
    'title.str.contains("Analy", case=False) & (title.str.contains("Modelling|Engineer|Systems|Architect", case=False)==False)'
)

Для удобства работы со скиллами разобьем вакансии по скиллам (появятся дубли вакансий по одному скиллу в записи)

In [63]:
clear_data = clear_data.explode('skills')

**Поиск дубликатов**

Для верности посмотрим дубликаты по ссылкам (для каждой вакансии она точно уникальная) в неочищенных данных.

In [64]:
data[data['link'].duplicated()]

,Unnamed: 0,html,title,link,candidats,geo,city,country,description,skills,company,workplace,date,get_size_sphere,size,sphere


Дублей нет

In [65]:
clear_data[clear_data.duplicated()]

,title,link,candidats,city,country,skills,company,workplace,date,size,sphere


Логично, что и тут дублей нет

**Итоговые данные**

In [66]:
clear_data.shape

(974, 11)

Получилось 974 записи. Помним, что одна вакансия разбита на несколько записей по одному хард скиллу из списка скиллов.

In [67]:
print(f'Всего уникальных вакансий в датафрейме: {clear_data["link"].nunique()}')

Всего уникальных вакансий в датафрейме: 356


In [68]:
clear_data.to_csv('linkedin_da.csv', index = False)